In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from scripts import request, table, utils

project_path = os.path.abspath(os.path.expandvars("$GITHUB_HOME/claim-extraction"))
experiments_path = os.path.join(project_path, "experiments")

In [2]:
connection_infos = request.extract_infos('../private.json')

msgs_base_path = os.path.join(project_path, 'messages/CS')

msgs_file_paths = {
    'system_1':  f'{msgs_base_path}/system_1.txt',
    'system_2':  f'{msgs_base_path}/system_2.txt',
    'user_1':    f'{msgs_base_path}/user_1.txt',
    'user_2':    f'{msgs_base_path}/user_2.txt',
    'assistant': f'{msgs_base_path}/assistant.txt'
}

output_dir = os.path.join(experiments_path, f'output/cs_dataset')
utils.check_path(output_dir)

tables_file_path = os.path.join(experiments_path, 'extracted_tables/CS_Dataset.json')

answers_path = os.path.join(output_dir, "answers")

In [ ]:
num_thread = 40

while True:
    tables_to_process = table.check_processed_tables(tables_file_path, answers_path)
    if tables_to_process == 0:
        break

    tables = table.load_tables_from_json(tables_file_path)
    request.run(connection_infos, msgs_file_paths, tables, output_dir, min(num_thread, tables_to_process))

In [4]:
answers_path = os.path.join(output_dir, "answers")
table.check_processed_tables(tables_file_path, answers_path)